In [1]:
import os,math,sys, numpy as np, pandas as pd
import scipy
from scipy import linalg
from numpy import dot
from collections import defaultdict
from sklearn.cluster import KMeans
import glob
import matplotlib.pyplot as plt
from scipy.spatial import distance

In [2]:
##First subcluser name
filetype='{0}'.format(sys.argv[1])
filetype = input('What is the datatype name? ')

What is the datatype name? Review_hMor


In [3]:
#Second name
filetype2='{0}'.format(sys.argv[1])
filetype2 = input('What is the datatype name? ')

What is the datatype name? compoundStructure


In [8]:
###Read in first file
filename='../SEM/%s/TotalFrequencyMatrix_Average.txt'%(filetype)
temp_df=pd.read_csv(filename, sep='\t',header=0,index_col=0)
temp_df.drop(list(temp_df)[-1], inplace=True, axis=1)
temp_df.head()

,Buprenorphine,DAMGO,Endomorphine1,Fentanyl,Loperamide,Meptazinol,Met-Enk,Morphine,Oxycodone,Tramadol,...,Cmp 7,Cmp 8,Cmp 9,Cmp 10,Cmp 11,Cmp 12,Cmp 13,Cmp 14,Cmp 15,Cmp 16
Buprenorphine,1.000000,0.001021,0.003125,0.028854,0.000667,0.578083,0.001042,0.062937,0.071875,0.444875,...,0.000958,0.005104,0.833604,0.106729,0.559063,0.063521,0.001375,0.623854,0.530042,0.142792
DAMGO,0.001021,1.000000,0.475167,0.162729,0.754521,0.001208,0.868667,0.124063,0.113667,0.001000,...,0.822396,0.402354,0.001104,0.091042,0.001208,0.125562,0.738604,0.001417,0.000604,0.044896
Endomorphine1,0.003125,0.475167,1.000000,0.532833,0.536646,0.005917,0.457646,0.487604,0.466271,0.014063,...,0.428021,0.579000,0.003875,0.389896,0.021771,0.471271,0.618854,0.014958,0.008812,0.400500
Fentanyl,0.028854,0.162729,0.532833,1.000000,0.182312,0.037583,0.162688,0.919708,0.879250,0.062833,...,0.160312,0.411917,0.029188,0.684083,0.079792,0.861313,0.232646,0.049021,0.046250,0.829792
Loperamide,0.000667,0.754521,0.536646,0.182312,1.000000,0.001125,0.695688,0.139937,0.129167,0.006458,...,0.755188,0.514563,0.000687,0.118896,0.002083,0.160792,0.751667,0.001729,0.004812,0.057813


In [5]:
list(temp_df)

['Buprenorphine',
 'DAMGO',
 'Endomorphine1',
 'Fentanyl',
 'Loperamide',
 'Meptazinol',
 'Met-Enk',
 'Morphine',
 'Oxycodone',
 'Tramadol',
 'Cmp 1',
 'Cmp 3',
 'Cmp 4',
 'Cmp 5',
 'Cmp 6',
 'Cmp 7',
 'Cmp 8',
 'Cmp 9',
 'Cmp 10',
 'Cmp 11',
 'Cmp 12',
 'Cmp 13',
 'Cmp 14',
 'Cmp 15',
 'Cmp 16']

In [11]:
totalchangearray=[]

path='../SEM/RandomClustering/%s/FrequencyMatrix/'%(filetype2)
for filename2 in glob.glob(os.path.join(path,'FrequencyMatrix_*Average.txt')):


    ###Read in second file

    temp_df2=pd.read_csv(filename2, sep='\t',header=0,index_col=0)
    temp_df2.drop(list(temp_df2)[-1], inplace=True, axis=1)


    ##Compounds found in both the two datasets for comparison
    shared_compounds=[i for i in temp_df2.index.values if i in temp_df.index.values]


    ##Data matrix using only data from shared compounds
    simdict1=temp_df.loc[shared_compounds,shared_compounds].values
    simdict2=temp_df2.loc[shared_compounds,shared_compounds].values


    ###The general idea here is this: We have calculated a threshold value seperating comps
    ##in the same cluster vs those in a different cluster.
    ##Using this value, we will noew calculate  the distance between pairs of compounds i,j 
    ##In matrix 1 and matrix 2. If EDi,jmatrix1< thresh and  EDi,jmatrix2 <thresh we can 
    ##Conclude that these two compounds are in the same cluster in both data types.

    ##Dictionary of threshold values for each subtype
    threshdict={'4receptors':1.2,'hDor_Beta':0.75,'hDor_Gprot':1.25,'hDor_Gproteins':1.2,'hDor_Kir_CAMP':1.25,'hDor':1.1,'hMor_All4':1.3,'hMor_Beta':0.9,'hMor_CAMP_GAI2':1.4,'hMor_CAMP':1.6,'hMor_GAI2':1.4,'hMor_Gprot':1.4,'hMor_GRK2':1.2,'hMor_GRK6':1.0,'hMor_GRK26_GAI2':1.1,'hMor_GRK26':1.0,'hMor':1.2,'Pfizer_BRET':1.25,'Pfizer':1.4,'rMor':1.2}
    #thresh=threshdict[filetype]
    thresh=1.1
    print('The threshold is',thresh)


    ##Calculate Euclidean Distance between pairs of compounds within each matrix
    changearray=[0,0,0,0]

    for pos1, compound1 in enumerate (shared_compounds): #For every compound in matrix
        for pos2, compound2 in enumerate (shared_compounds):#for Every compound in matrix
            #print(compound1,simdict1[pos1])
            #print(compound2,simdict1[pos2])
            #print(compound1,simdict2[pos1])
            #print(compound2,simdict2[pos2])


            tempdistance=distance.euclidean(np.array(simdict1[pos1]),
                                                            np.array(simdict1[pos2]))
            tempdistance2=distance.euclidean(np.array(simdict2[pos1]),np.array(simdict2[pos2]))

            if tempdistance<thresh:#same cluster for dict1
                if tempdistance2<thresh:
                    changearray[0]+=1 #same cluster in both dicts
                else:
                    changearray[1]+=1#different in dict2
            else:
                if tempdistance2<thresh:#different in dict 1 but same in dict2
                    changearray[2]+=1
                else:
                    changearray[3]+=1##different in both

    print('changearray',changearray)

    ##Finally, we are interested in knowing how clusters compare for dict1 and dict2
    ##There are two ways clustering is the same: compounds in the same cluster stay in the 
    ##same cluster and compounds in different clusters stay in different clusters.
    ##Therefore, I combine same same with diff diff, and combine same diff with diff same
    changearray=[changearray[0]+changearray[3],changearray[1]+changearray[2]]
    #print('#Same %s \n#Diff %s'%(changearray[0],changearray[1]))
    #print('Fraciton Same %s \nFraction Diff %s'
    #      %(changearray[0]/float(sum(changearray)),changearray[1]/float(sum(changearray))))
    
    
    
    totalchangearray.append(changearray[1]/float(sum(changearray)))

totalchangearray=np.array(totalchangearray)




In [30]:
totalchangearray

array([0.2944, 0.3456, 0.3168, 0.6304, 0.4192, 0.3232, 0.3808, 0.3264,
       0.3872, 0.3584, 0.432 , 0.368 , 0.3328, 0.3424, 0.6752, 0.288 ,
       0.2528, 0.3168, 0.2848, 0.6976, 0.3008, 0.3904, 0.3744, 0.2496,
       0.3776, 0.2656, 0.336 , 0.24  , 0.3264, 0.3008, 0.352 , 0.288 ,
       0.7328, 0.304 , 0.288 , 0.2624, 0.3008, 0.4064, 0.3584, 0.3776,
       0.3264, 0.3648, 0.3264, 0.2496, 0.2848, 0.2848, 0.3168, 0.3008,
       0.4224, 0.3456])

In [20]:
outfile=open('../SEM/RandomClustering/%s/DifferencesfromRandom_%s_%s.txt'%(filetype2,filetype,filetype2),'w')
outfile.write('mean=%s'%(np.mean(totalchangearray)))
outfile.write('\nstd=%s'%(np.std(totalchangearray)))
for i in totalchangearray:
    outfile.write('\n%s'%(str(i)))

In [ ]:
Buprenorphine
DAMGO
Endomorphin1
Fentanyl
Loperamide
Meptazinol
Met-Enkephalin
Morphine
Oxycodone
Tramadol
Compound-1
Compound-3
Compound-4
Compound-5
Compound-6
Compound-7
Compound-8	Compound-9	Compound-10	Compound-11	Compound-12	Compound-13	Compound-14	Compound-15	Compound-16


In [ ]:
['Buprenorphine',
 'DAMGO',
 'Endomorphine1',
 'Fentanyl',
 'Loperamide',
 'Meptazinol',
 'Met-Enk',
 'Morphine',
 'Oxycodone',
 'Tramadol',
 'Cmp 1',
 'Cmp 3',
 'Cmp 4',
 'Cmp 5',
 'Cmp 6',
 'Cmp 7',
 'Cmp 8',
 'Cmp 9',
 'Cmp 10',
 'Cmp 11',
 'Cmp 12',
 'Cmp 13',
 'Cmp 14',
 'Cmp 15',
 'Cmp 16']

In [17]:
sharedcompounds=list(temp_df)

In [28]:
totalchangearray=[]


path='../SEM/compoundStructure/Clusters/Random/'
outercounter=0
for filename2 in glob.glob(os.path.join(path,'Frequencymatrix*')):
    if outercounter<100:


        ###Read in second file

        temp_df2=pd.read_csv(filename2, sep='\t',header=0,index_col=0)
        temp_df2.drop(list(temp_df2)[-1], inplace=True, axis=1)


        ##Compounds found in both the two datasets for comparison
        shared_compounds=[i for i in temp_df.index.values if i in temp_df.index.values]
        #print(shared_compounds)

        ##Data matrix using only data from shared compounds
        simdict1=temp_df.values
        simdict2=temp_df2.values


        ###The general idea here is this: We have calculated a threshold value seperating comps
        ##in the same cluster vs those in a different cluster.
        ##Using this value, we will noew calculate  the distance between pairs of compounds i,j 
        ##In matrix 1 and matrix 2. If EDi,jmatrix1< thresh and  EDi,jmatrix2 <thresh we can 
        ##Conclude that these two compounds are in the same cluster in both data types.

        ##Dictionary of threshold values for each subtype
        threshdict={'4receptors':1.2,'hDor_Beta':0.75,'hDor_Gprot':1.25,'hDor_Gproteins':1.2,'hDor_Kir_CAMP':1.25,'hDor':1.1,'hMor_All4':1.3,'hMor_Beta':0.9,'hMor_CAMP_GAI2':1.4,'hMor_CAMP':1.6,'hMor_GAI2':1.4,'hMor_Gprot':1.4,'hMor_GRK2':1.2,'hMor_GRK6':1.0,'hMor_GRK26_GAI2':1.1,'hMor_GRK26':1.0,'hMor':1.2,'Pfizer_BRET':1.25,'Pfizer':1.4,'rMor':1.2}
        #thresh=threshdict[filetype]
        thresh=1.7
        print('The threshold is',thresh)


        ##Calculate Euclidean Distance between pairs of compounds within each matrix
        changearray=[0,0,0,0]

        for pos1, compound1 in enumerate (shared_compounds): #For every compound in matrix
            for pos2, compound2 in enumerate (shared_compounds):#for Every compound in matrix
                #print(compound1,simdict1[pos1])
                #print(compound2,simdict1[pos2])
                #print(compound1,simdict2[pos1])
                #print(compound2,simdict2[pos2])


                tempdistance=distance.euclidean(np.array(simdict1[pos1]),
                                                                np.array(simdict1[pos2]))
                tempdistance2=distance.euclidean(np.array(simdict2[pos1]),np.array(simdict2[pos2]))

                if tempdistance<thresh:#same cluster for dict1
                    if tempdistance2<thresh:
                        changearray[0]+=1 #same cluster in both dicts
                    else:
                        changearray[1]+=1#different in dict2
                else:
                    if tempdistance2<thresh:#different in dict 1 but same in dict2
                        changearray[2]+=1
                    else:
                        changearray[3]+=1##different in both
        print(outercounter)
        print('changearray',changearray)

        ##Finally, we are interested in knowing how clusters compare for dict1 and dict2
        ##There are two ways clustering is the same: compounds in the same cluster stay in the 
        ##same cluster and compounds in different clusters stay in different clusters.
        ##Therefore, I combine same same with diff diff, and combine same diff with diff same
        changearray=[changearray[0]+changearray[3],changearray[1]+changearray[2]]
        #print('#Same %s \n#Diff %s'%(changearray[0],changearray[1]))
        #print('Fraciton Same %s \nFraction Diff %s'
        #      %(changearray[0]/float(sum(changearray)),changearray[1]/float(sum(changearray))))



        totalchangearray.append(changearray[1]/float(sum(changearray)))
    outercounter+=1

totalchangearray=np.array(totalchangearray)





The threshold is 1.7
0
changearray [123, 120, 102, 280]
The threshold is 1.7
1
changearray [87, 156, 124, 258]
The threshold is 1.7
2
changearray [95, 148, 112, 270]
The threshold is 1.7
3
changearray [99, 144, 148, 234]
The threshold is 1.7
4
changearray [89, 154, 124, 258]
The threshold is 1.7
5
changearray [97, 146, 104, 278]
The threshold is 1.7
6
changearray [99, 144, 126, 256]
The threshold is 1.7
7
changearray [97, 146, 142, 240]
The threshold is 1.7
8
changearray [99, 144, 98, 284]
The threshold is 1.7
9
changearray [101, 142, 160, 222]
The threshold is 1.7
10
changearray [99, 144, 126, 256]
The threshold is 1.7
11
changearray [73, 170, 116, 266]
The threshold is 1.7
12
changearray [101, 142, 142, 240]
The threshold is 1.7
13
changearray [97, 146, 150, 232]
The threshold is 1.7
14
changearray [83, 160, 118, 264]
The threshold is 1.7
15
changearray [87, 156, 126, 256]
The threshold is 1.7
16
changearray [101, 142, 134, 248]
The threshold is 1.7
17
changearray [103, 140, 118, 264

In [30]:
totalchangearray

array([0.3552, 0.448 , 0.416 , 0.4672, 0.4448, 0.4   , 0.432 , 0.4608,
       0.3872, 0.4832, 0.432 , 0.4576, 0.4544, 0.4736, 0.4448, 0.4512,
       0.4416, 0.4128, 0.3968, 0.3808, 0.4448, 0.384 , 0.4352, 0.4064,
       0.4256, 0.464 , 0.4832, 0.4352, 0.4256, 0.432 , 0.4352, 0.4544,
       0.416 , 0.3904, 0.4096, 0.4096, 0.4288, 0.4224, 0.432 , 0.4768,
       0.4032, 0.4704, 0.4224, 0.4192, 0.4224, 0.4736, 0.4352, 0.4128,
       0.3808, 0.4256, 0.448 , 0.4352, 0.3904, 0.4416, 0.4064, 0.448 ,
       0.4256, 0.4608, 0.4672, 0.4096, 0.448 , 0.4672, 0.432 , 0.4512,
       0.4416, 0.448 , 0.4192, 0.4416, 0.4   , 0.4128, 0.464 , 0.464 ,
       0.4416, 0.4576, 0.4032, 0.448 , 0.48  , 0.448 , 0.4288, 0.4288,
       0.4256, 0.4288, 0.4608, 0.448 , 0.4416, 0.4256, 0.4512, 0.464 ,
       0.4672, 0.4384, 0.5024, 0.4672, 0.4192, 0.3936, 0.4512, 0.3936,
       0.4128, 0.4544, 0.4512, 0.448 ])

In [31]:
outfile=open('../SEM/compoundStructure/Clusters/DifferencesfromRandom_%s_%s.txt'%(filetype,filetype2),'w')
outfile.write('mean=%s'%(np.mean(totalchangearray)))
outfile.write('\nstd=%s'%(np.std(totalchangearray)))
for i in totalchangearray:
    outfile.write('\n%s'%(str(i)))
    
outfile=open('../SEM/compoundStructure/Clusters/DifferencesfromRandom_%s_%s.txt'%(filetype,filetype2),'w')
outfile.write('mean=%s'%(np.mean(totalchangearray)))
outfile.write('\nstd=%s'%(np.std(totalchangearray)))
for i in totalchangearray:
    outfile.write('\n%s'%(str(i)))